<a href="https://colab.research.google.com/github/diegoax/ALNAE-2025/blob/main/notebooks/clase22_ALNAE_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clase 22 (miércoles 17 de junio, 2025)
---

# Reducción Rápida de Dimensionalidad mediante Proyección Aleatoria | Lema de Johnson–Lindenstrauss


## Introducción

En problemas de alta dimensión, como el procesamiento de datos, aprendizaje automático o compresión, a menudo necesitamos reducir la dimensión de los datos sin perder demasiada información. Una forma sorprendentemente efectiva de hacerlo es mediante **proyecciones aleatorias**.

Este método se basa en una idea central: **en alta dimensión, los vectores aleatorios son casi ortogonales**. Esto permite que proyecciones simples, definidas con matrices aleatorias, puedan preservar aproximadamente las distancias entre puntos.

---



Imaginemos que necesitamos una reducción de dimensionalidad muy rápida, mediante una transformación o proyección desde un espacio de dimensión $m$ a un espacio de menor dimensión $k$.

- ¡No disponemos de todos los datos ahora; no podemos calcular una matriz de covarianzas ni de disimilitudes!
- ¡No podemos permitirnos calcular una transformación ortogonal, eso es demasiado costoso computacionalmente!

¿Podemos simplemente transformar las muestras de entrenamiento a un espacio de menor dimensión usando una **transformación aleatoria**?




## 🔁 Proyecciones y Matrices de Proyección

Dada una matriz $J \in \mathbb{R}^{k \times m}$ (con $k < m$), podemos definir una proyección de vectores de $\mathbb{R}^m$ a $\mathbb{R}^k$ como $x \mapsto Jx$.

Una matriz de proyección asociada se define como:

$$
P = J^\top J \in \mathbb{R}^{m \times m}
$$

Cuando las filas de $J$ son ortonormales, se tiene que:

- $J J^\top = I_k$
- $P = J^\top J$ es **simétrica** ($P = P^\top$)
- $P$ es **idempotente** ($P^2 = P$)

Esto significa que $P$ proyecta ortogonalmente sobre el subespacio imagen de $J^\top$.

---

## 🎲 Proyecciones Aleatorias

Supongamos ahora que cada entrada de $J$ se elige al azar, por ejemplo:

$$
J_{ij} \sim \mathcal{N}(0, 1/k)
$$

Es decir, cada componente se toma de una distribución normal con media cero y varianza $1/k$. En este caso, se puede demostrar que:

$$
\mathbb{E}[J^\top J] = I_m
$$

Esto implica que:

- En promedio, las columnas de $J$ son ortogonales
- Las longitudes se preservan en promedio:
  
$$
\mathbb{E}\|Jx\|^2 = \|x\|^2
$$


(Observar que $J$ tiene generalmente rango $k$, y por lo que $J^\top J$ también tiene rango $k$, pero su esperanza tiene rango $m$. Da para pensar...)


---

## 📐 Preservación de Distancias

Una motivación clave es preservar distancias entre puntos $x_i$ y $x_j$ después de proyectar. Queremos que:

$$
\|Jx_i - Jx_j\|^2 \approx \|x_i - x_j\|^2
$$

De lo anterior sabemos que para un par de puntos, eso se puede hacer (al menos en promedio). La dificultad ocurre que quiero preservar distancias pero entre muchos puntos. Observar que ya en $\mathbb R^2$, tres puntos no alineados no pueden preservar su distancia mediante una proyección ortogonal en un subespacio de dimensión $1$.

---

## 📏 El Lema de Johnson–Lindenstrauss

El siguiente es un resultado famoso que da una respuesta positiva a nuestro problema si agregamos cierta tolerancia.

> Dado un conjunto de $n$ puntos $x_1, \dots, x_n$ en $\mathbb{R}^m$ y un error $\epsilon \in (0,1)$, si:
>
> $$
> k \geq \frac{8 \log n}{\epsilon^2}
> $$
>
> entonces existe una proyección lineal $J : \mathbb{R}^m \to \mathbb{R}^k$ tal que para todos los pares $i,j$:
>
> $$
> (1 - \epsilon) \|x_i - x_j\|^2 \leq \|Jx_i - Jx_j\|^2 \leq (1 + \epsilon) \|x_i - x_j\|^2
> $$

Esto implica que **se pueden reducir las dimensiones drásticamente manteniendo la estructura geométrica** del conjunto de puntos.

Comentarios:
- Observar que la dimensión $k$ requerida sólo depende de la cantidad de puntos y no del tamaño del espacio inicial.
- La proyección se puede construir independiente de los datos.


---

## 🧪 Ejemplo en Julia

Veamos un ejemplo práctico para ilustrar el lema.



In [5]:
using Random, LinearAlgebra, Statistics

# Número de puntos y dimensión original
n, m = 100, 1000
X = randn(m, n)  # Cada columna es un punto en R^m

# Parámetro de error deseado
ϵ = 0.2
k = ceil(Int, 8 * log(n) / ϵ^2)

# Matriz de proyección aleatoria
J = randn(k, m) / sqrt(k)
Y = J * X  # Puntos proyectados en R^k

# Función para medir la distorsión relativa promedio
function distortion(X, Y)
    n = size(X, 2)
    d_orig = Float64[]
    d_proj = Float64[]
    for i in 1:n-1, j in i+1:n
        push!(d_orig, norm(X[:, i] - X[:, j])^2)
        push!(d_proj, norm(Y[:, i] - Y[:, j])^2)
    end
    rel_error = abs.(d_proj .- d_orig) ./ d_orig
    return mean(rel_error)
end

println("Proyectando a dimensión k = $k")
println("Distorsión relativa promedio: ", distortion(X, Y))


Proyectando a dimensión k = 922
Distorsión relativa promedio: 0.03544154082506118


### Más detalles

- Se utiliza cuando los datos tienen una dimensión tan alta que es demasiado costoso realizar el cálculo.

- Cuando la dimensionalidad es alta y el número de muestras es demasiado bajo como para calcular covarianzas de forma confiable.

- Cuando no se tiene acceso al conjunto de datos completo (por ejemplo, en datos en tiempo real), la proyección se establece **sin ver los datos**:
  - alternativa no adaptativa  
  - no es necesario calcular todas las disimilitudes por pares por adelantado
  - es muy barato y rápido computacionalmente.


# Rank one perturbation: Sherman-Morrison-Woodbury formula

Cómo varía la inversa de una matriz cuando a esta se le perturba? Comencemos con el caso más sencillo de hacer una perturabación, por matrices de rango $1$ de la identidad:

Qué podemos decir de
$$
(\textrm{Id}-uv^T)^{-1}
$$

Sea $B=\textrm{Id}-uv^T$. Observar que:
- $B$ restringido a $v^\perp$ es la identidad;
-$Bu= u-v^Tu u=(1-v^Tu)u$

Luego tenemos que si $B$ es invertible (que sucede si $v^Tu\neq 1$) (*porqué?),
- $B^{-1}$ restringido a $v^\perp$ es la identidad
- $B^{-1}u=\frac{1}{(1-v^Tu)}u$
Esto sugiere que la inversa $B^{-1}$ es de la forma $\textrm{Id}-\alpha uv^T$.

Un cálculo sencillo muestra que
$$
(\textrm{Id}-uv^T)^{-1} = \textrm{Id}+\frac{1}{(1-v^Tu)} uv^T.
$$

Una forma similar ocurre si en vez de considerar una perturbaciń por matrices de rango uno, tomamos una perturbación tipo $UV^T$.  Todo esto es un caso particular de la siguiente fórmula:


**Formula de Sherman-Morrison-Woodbury: **
$$
(A-UV^T)^{-1}=A^{-1} + A^{-1}U(\textrm{Id}-V^TA^{-1}U)^{-1}V^TA^{-1}.
$$

Veamos una aplicación de esto.

## Actualización de Mínimos Cuadrados


Considere la ecuación normal de resolve el problema de mínimos cuadrados $\|Ax-b\|^2$:

$$
A^T A \hat{x} = A^T b
$$

Supongamos que llega una nueva fila $r$ (de tamaño $1 \times n$) y un nuevo dato $b_{m+1}$. Entonces, el sistema extendido se escribe como:

$$
\begin{bmatrix}
A^T \\
r^T
\end{bmatrix}^T
\begin{bmatrix}
A & r^T
\end{bmatrix}
\hat{x}_{\text{new}} =
\begin{bmatrix}
A^T \\
r^T
\end{bmatrix}^T
\begin{bmatrix}
b \\
b_{m+1}
\end{bmatrix}
$$

o simplemente:

$$
(A^T A + r^T r) \hat{x}_{\text{new}} = A^T b + r^T b_{m+1}
$$

La nueva matriz de normales es $A^T A + r^T r$, que es una **perturbación de rango 1** de $A^T A$.

Para evitar recalcular todo, se usa la siguiente fórmula de actualización (Sherman–Morrison):

$$
(A^T A + r^T r)^{-1} =
(A^T A)^{-1} - \frac{(A^T A)^{-1} r^T r (A^T A)^{-1}}{1 + r (A^T A)^{-1} r^T}
$$

Sea $c = \frac{1}{1 + r (A^T A)^{-1} r^T}$. Entonces:

$$
(A^T A + r^T r)^{-1}
= (A^T A)^{-1} - c (A^T A)^{-1} r^T r (A^T A)^{-1}
$$

Para encontrar $c$, solo se necesita resolver $y = (A^T A)^{-1} r^T$.

> Esta técnica permite actualizar la solución $\hat{x}_{\text{new}}$ sin recalcular todo el sistema. Si llegan $M$ nuevas filas en lugar de una, se puede aplicar de forma recursiva: **Mínimos Cuadrados Recursivos**.
> Este tipo de análisis está relacionado al filtro de Kalman, que tiene como versión particular a lo recién visto, y es utilizado por ejemplo para el GPS.